In [1]:
!pip install summa
!pip install datasets
!pip install tqdm
!pip install evaluate
!pip install sacrebleu
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 1.8 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=0efb2b0479ab135c6c8ca1e48bb99bdb165f7d162b937a43bd318697f8ba3217
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.9 MB/s 
     |████████████████████████████████| 182 kB 6.3 MB/s 
     |████████████████████████████████| 115 kB 45.9 MB/s 
     |████████████████████████████████| 212 kB 47.1 MB/s 
     |████████████████████████████████| 127 kB 42.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes:

In [2]:
import os
import json
import nltk
from nltk.tokenize import sent_tokenize
import matplotlib.pyplot as plt
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATA_PATH = '/content/drive/MyDrive/nlp-project/data'
BILLSUM_DATA_PATH = os.path.join(DATA_PATH, 'billsum_v4_1')

In [5]:
from datasets import load_dataset

In [6]:
TRAIN_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_train_data_final_OFFICIAL.jsonl")
TEST_DATA_FILE = os.path.join(BILLSUM_DATA_PATH, "us_test_data_final_OFFICIAL.jsonl")

dataset = load_dataset("json", data_files={
    "train": TRAIN_DATA_FILE,
    "test": TEST_DATA_FILE,
})

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0a257b7a118a3705/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def preprocess_text(row):
    text = row['text']
    text = text.split('\n')
    text = list(map(lambda x: x.strip(), text))
    text = list(filter(lambda x: True if x else False, text))
    text = " ".join(text)
    row['text'] = text
    return row

In [8]:
dataset = dataset.map(preprocess_text)

  0%|          | 0/18949 [00:00<?, ?ex/s]

  0%|          | 0/3269 [00:00<?, ?ex/s]

In [9]:
from summa.summarizer import summarize
from tqdm import tqdm
import evaluate

In [15]:
def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def evaluate_summaries(dataset, metric, batch_size=16, column_text="text", column_summary="summary"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)
    ):
        predicted_sums = []
        for text in article_batch:
            predicted = summarize(text, words=100)
            predicted_sums.append(predicted)
        metric.add_batch(predictions=predicted_sums, references=target_batch)
    
    score = metric.compute()
    return score

In [16]:
rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

score = evaluate_summaries(dataset['test'], rouge_metric)

100%|██████████| 205/205 [01:29<00:00,  2.28it/s]


In [17]:
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"TextRank"])

,rouge1,rouge2,rougeL,rougeLsum
TextRank,0.308503,0.142209,0.200091,0.201106
